# Spurious Correlations of Highly Dimensional Big Data

This Notebook aims at showing how PCA and random projection can solve the problem of spurious correlations in Big Data.

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import random_projection
from sklearn.utils import shuffle

## DataFrame Creation

In [2]:
# User defined parameters

# Number of rows for df1
x = 10000

# Number of columns for df1
y = 100000

# Number of rows for df2
z = 100

In [3]:
np.random.seed(0)

In [ ]:
# Creating a dataframe with x number of rows and y number of columns
df = pd.DataFrame(np.random.random_sample((x,y)))

# Shuffle rows of dataframe
df = df.sample(frac=1)

In order to assess the correlations of the different parameters, correlations between the column with index 0 and the 999 other first columns is assessed.

In [ ]:
# Assigning X to all columns except 0
X_df = df.drop(columns=0)
X_df.head()

# Assigning Y to column 0
Y_df = df[0]
print(Y_df)

# The following line makes Y become a list
Y_df = np.array(Y_df).reshape(-1)
print(X_df.shape,Y_df.shape)

# Calculate correlations between selected column and column 0
list_titles = X_df.columns
list_corr_df1 = []
for i in list_titles[0:z]:
    list_corr_df1.append(abs(np.corrcoef(Y_df, X_df[i])[0][1]))

To see if indeed the correlations between the different parameters increases with the data size, the original dataframe is compared to a sub-set dataframe which only takes the first z rows of the original dataframe. If the correlations in the original dataframe are higher than in the smaller dataframe this would prove that the bigger the data size the more frequent the number of spurious correlations.

In [ ]:
# Creating smaller dataframe taking z number of rows from original dataframe
df2 = df.iloc[:z]
df2

In [ ]:
# Assigning X to all columns except 0
X_df2 = df2.drop(columns=0)
X_df2.head()

# Assigning Y to column 0
Y_df2 = df2[0]
print(Y_df2)

# The following line makes Y become a list
Y_df2 = np.array(Y_df2).reshape(-1)
print(X_df2.shape,Y_df2.shape)

list_titles = X_df2.columns

# Calculate correlations between selected column and column 0
list_corr_df2 = []
for i in list_titles[0:z]:
    list_corr_df2.append(abs(np.corrcoef(Y_df2, X_df2[i])[0][1]))

We now compare which of the correlation lists has the highest numbers of every parameter

In [ ]:
a = 0
b = 0
for i in range(0,len(list_corr_df1)):
    if abs(list_corr_df1[i]) > 0.01:
        a+=1
    else:
        pass
        
for i in range(0,len(list_corr_df2)):
    if abs(list_corr_df2[i]) > 0.01:
        b+=1
    else:
        pass

print("Percentage of correlations in df1:",(a/len(list_corr_df1)*100),"%")

print("Percentage of correlations in df2:",(b/len(list_corr_df2)*100),"%")

Clearly it is seen that the lower the number of observations, the higher the frequency of spurious correlations. However the number of spurious correlations present in the bigger DataFrame is still very significant.

To counter spurious correlations, Principal Component Analysis and Random Projection can be used.

## Principal Component Analysis

In PCA, it is up to the user to choose how many dimension the final data has. This is because the explained variance tells you how much information (variance) can be attributed to each of the principal components. This is important as while you can convert a y dimensional space to 2 dimensional space, you lose some of the variance (information) when you do this. It is up to the user to decide how much variance he is ready to lose. 

In [ ]:
# Here we pass the daframe of y columns in 2 dimensions
pca = PCA(n_components=2)
df_pca = pd.DataFrame(pca.fit_transform(df))

In [ ]:
# How much of the information is kept by the new dataframe?
print(pca.explained_variance_ratio_)

The new dataframe is not useful as it almost does not capture any variance. Thus we need more dimensionss.

In [ ]:
# Here we pass the daframe of y columns in 2 dimensions
pca = PCA(n_components=100)
df_pca = pd.DataFrame(pca.fit_transform(df))
print(pca.explained_variance_ratio_)

In [ ]:
# Here we pass the daframe of y columns in 2 dimensions
pca = PCA(n_components=1000)
df_pca = pd.DataFrame(pca.fit_transform(df))
print(pca.explained_variance_ratio_)

In [ ]:
df_new = df_pca
# Assigning X to all columns except 0
X_df_new = df_new.drop(columns=0)

# Assigning Y to column 0
Y_df_new = df_new[0]
print(Y_df_new)

# The following line makes Y become a list
Y_df_new = np.array(Y_df_new).reshape(-1)
print(X_df_new.shape,Y_df_new.shape)

list_titles = X_df_new.columns

# Calculate correlations between selected column and column 0
list_corr_df_new = []
for i in list_titles[0:len(list_titles)]:
    list_corr_df_new.append(abs(np.corrcoef(Y_df_new, X_df_new[i])[0][1]))

In [ ]:
a = 0
b = 0
for i in range(0,len(list_corr_df1)):
    if abs(list_corr_df1[i]) > 0.01:
        a+=1
    else:
        pass
        
for i in range(0,len(list_corr_df_new)):
    if abs(list_corr_df_new[i]) > 0.01:
        b+=1
    else:
        pass

print("Percentage of correlations in df1:",(a/len(list_corr_df1)*100),"%")

print("Percentage of correlations in df_new:",(b/len(list_corr_df_new)*100),"%")

## Random Projection

In Random Projection, the parameter eps defines how much the Random Projection can deviate from original DataFrame.
The lower the percentage of the parameter the higher the fidelity of the transformed data.
However the higher the fidelity the less the reduction of dimensions.

In [ ]:
transformer = random_projection.GaussianRandomProjection(eps = 0.1)
df_rp = pd.DataFrame(transformer.fit_transform(df))
df_rp

In [ ]:
df_new = df_rp
# Assigning X to all columns except 0
X_df_new = df_new.drop(columns=0)

# Assigning Y to column 0
Y_df_new = df_new[0]
print(Y_df_new)

# The following line makes Y become a list
Y_df_new = np.array(Y_df_new).reshape(-1)
print(X_df_new.shape,Y_df_new.shape)

list_titles = X_df_new.columns

list_corr_df_new = []
for i in list_titles[0:len(list_titles)]:
    print(i)
    list_corr_df_new.append(abs(np.corrcoef(Y_df_new, X_df_new[i])[0][1]))
    print('Correlation matrix for column 0 and and column' + str(i) + ': ' + str(np.corrcoef(Y_df_new, X_df_new[i])[0][1]))

In [ ]:
a = 0
b = 0
for i in range(0,len(list_corr_df1)):
    if abs(list_corr_df1[i]) > 0.01:
        a+=1
    else:
        pass
        
for i in range(0,len(list_corr_df_new)):
    if abs(list_corr_df_new[i]) > 0.01:
        b+=1
    else:
        pass

print("Percentage of correlations in df1:",(a/len(list_corr_df1)*100),"%")

print("Percentage of correlations in df_new:",(b/len(list_corr_df_new)*100),"%")